<div style="text-align: center;">
    <h1>BTC Perpetuals Risk Quantification Pipeline</h1>
</div>

# 1. Introduction & Data Fetching

This project models and predicts funding rates for perpetual contracts using Monte Carlo simulations. It employs Merton’s jump diffusion for index prices and the Ornstein-Uhlenbeck process for funding rates to derive metrics like expected liquidation time and probability. These figures enhance risk management in perpetual contracts trading.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
import warnings
import glob
import time

path_colors = ['lightseagreen', 'red', 'orange', 'purple', 'tan', 'magenta', 'cyan', 'lightgreen', 'pink', 'lightsalmon']
start = time.time()

In [2]:
def get_data_per_coin(col, coin): 
    
    """
    Extract and preprocess data for a specific coin from a collection of CSV files containing the time series of Perpetuals market indicators.

    Args:
    - col (str): The column to extract from the CSV files. Possible values: 'OI', 'Funding', 'Basis', 'Volume', 'Index', 'Price'
    - coin (str): The specific coin or base currency to filter the data for.Possible values: 'BTC', 'ETH'

    Returns:
    - pandas.DataFrame: A DataFrame containing the extracted and processed data. Each column represents a market
      where the specified coin is traded, and the index represents the date.

    Note:
    - This function assumes that CSV files containing the data are stored in a directory named 'Data' and follow a specific naming convention.
    - The function extracts data for the specified base currency (coin) and resamples it to a daily frequency.
    """

    csv_files = glob.glob('Data\*.csv')

    dataframes = []

    for csv_file in csv_files:
        df = pd.read_csv(csv_file,parse_dates=True).rename(columns={'base_currency':'Base Currency','market':'Market','oi':'OI','funding':'Funding','basis':'Basis','volume':'Volume','index_price':'Index','price':'Price','symbol':'Symbol','date':'Date'})
        df=df.loc[df['Base Currency']==coin]
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date',inplace=True)
        df = df.resample('1D').first()
        dataframes.append(df[col].to_frame(name=(df['Market'].iloc[0])))

    return pd.concat(dataframes,axis=1)

In [3]:
IndexPrice = get_data_per_coin(col='Index',coin='BTC').iloc[:, 0]
IndexPrice.index = pd.to_datetime(IndexPrice.index)
Funding = get_data_per_coin(col='Funding',coin='BTC').drop(columns='BITMEX')

In [4]:
Featured = pd.read_csv('Contracts.csv').set_index('Exchange')
Featured

,Maximum Leverage,MMR,TF,MA,Symbols
Exchange,,,,,
BINANCE,125,0.4%,0.050%,0,BTCUSD_PERP
OKEX,125,0.4%,0.100%,-,BTC-USDT
BYBIT,100,0.5%,0.055%,-,BTCUSD
DERIBIT,50,1% - 0.005%*N/S0,0.050%,-,BTC-PERPETUAL


# 2. The Setup

Within the context of Bitcoin perpetual contract trading, this setup represents a specific trading scenario. The chosen entry date (`T0`) marks the beginning of the analysis, and the artificial maturity date (`T`) signifies the endpoint of the simulated trading period. The number of contracts purchased (`N`) and the leverage applied (`L`) are critical factors influencing the position's risk and potential returns. The trader's decision to take either a `position` (`Short` or `Long`) indicates their market outlook. Furthermore, the choice of `exchange` is essential, as different exchanges may offer varying terms and funding mechanisms. This setup serves as a foundational framework for conducting simulations and assessments of Bitcoin perpetual contract positions in the dynamic cryptocurrency market.

In [5]:
T0 = '2023-01-21'                                                                                # Position Entry Date
T = '2023-04-01'                                                                                 # Artificial Maturity
N = 1                                                                                            # Number of Purchased Contracts
L = 50                                                                                           # Leverage
Position = 'Short'                                                                               # Position Specification (Short or Long)
Exchange = 'OKEX'                                                                                # Exchange Specification (BINANCE, BYBIT, OKEX or DERIBIT)

MaxL = {'BINANCE':125, 'BYBIT':100, 'OKEX':125, 'DERIBIT':50}                                    # Maximum Allowed Leverage for Featured Contracts for each Exchange 
S0 = IndexPrice.loc[T0]                                                                          # Initial Index Price
num_sim = 2000                                                                                   # Number of Paths
num_steps = (datetime.strptime(T, '%Y-%m-%d') - datetime.strptime(T0, '%Y-%m-%d')).days          # Number of Timesteps
random_shocks = np.random.standard_normal(size=(num_steps, num_sim))                             # White Noise Process Causing the Random Shocks of the Market

Visualize = False                                                                                # Visualize the MC Simulation Paths
Display = False                                                                                  # Display Parameters Estimates

# 3. The Functions & Methods of the Pipeline 

In [6]:
def IQR_Cleaning(data, LowerMultiplier = 1.5, UpperMultiplier = 1.5, clamp = False):

    """
    Perform IQR (Interquartile Range) cleaning on data.

    Args:
        - data (pandas.Series): Input data.
        - lower_multiplier (float, optional): Multiplier for lower whisker. Defaults to 1.5.
        - upper_multiplier (float, optional): Multiplier for upper whisker. Defaults to 1.5.
        - clamp (bool, optional): Whether to clamp the data within whiskers. Defaults to False.

    Returns:
        - pandas.Series: Cleaned data.
    """
    
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1

    lower_whisker = q1 - LowerMultiplier * iqr
    upper_whisker = q3 + UpperMultiplier * iqr

    if clamp:
        return data.clip(lower_whisker, upper_whisker)

    outliers = data[(data < lower_whisker) | (data > upper_whisker)]
    data.loc[outliers.index] = np.nan

    return data.interpolate(method='linear')

## 3.1. Funding Rate ($r_t$) Simulation Using the Ornstein-Uhlenbeck Process

During the analysis of the global funding rate data (which is constructed in a linear fashion from the funding rates of different exchanges), a distinct mean-reverting pattern became evident. This observation justified the utilization of the Ornstein-Uhlenbeck process within the Monte Carlo framework, as it accurately captures such reverting behaviors.

The *`Ornstein-Uhlenbeck`* process is a stochastic differential equation used to model `mean-reverting behavior` in time series data. It is widely applied in finance, economics, and other fields to describe phenomena where values tend to return to a long-term mean or equilibrium value over time.

The OU process is represented by the following `stochastic differential equation`:

\begin{equation*}
dX_t = \theta(\mu - X_t)dt + \sigma dW_t
\end{equation*}

\begin{equation*}
dW_t \approx W_{t+dt}-W_t \sim N(0, dt)
\end{equation*}

Where:
- $X_t$ represents the process at time t, in our case it is the transformed funding rate $F_t$.
- $dX_t$ represents the infinitesimal change in the process at time t.
- $\theta$ is the mean-reversion rate, indicating how quickly the process reverts to the mean.
- $\mu$ is the long-term mean or equilibrium value.
- $\sigma$ is the volatility of the process.
- $dW_t$ represents a differential of a standard Wiener process (or Brownian motion) at time t, which represents the stochastic noise (or random shocks).

The mean-reversion term $~\theta(\mu - X_t)~$ ensures that the process tends to move towards its long-term mean μ when it deviates from it. As $X_t$ approaches $\mu$, the mean-reversion force weakens, allowing the process to fluctuate around the equilibrium value. This process also exhibits stationary increments in the form of the random shocks $~\sigma dW_t~$, which means that the changes in the process over time have constant statistical properties.

The reasons behind the choice of this process are :

1. ***`Stationarity:`*** The OU process assumes that the underlying time series is stationary, which means it has a constant mean, variance, and autocovariance over time. As we mentioned earlier after conducting tests, the transformed funding rate data is stationary and that aligns well with the assumptions of the OU process, making it a suitable choice.

2. ***`Mean Reversion:`*** The mean reversion estimate is positive and indicates that it tends to revert back to a long-term mean value. The OU process is specifically designed to capture mean-reverting behavior, and the presence of mean reversion in your data further supports the appropriateness of the OU model.

3. ***`Economic & Theoretical Justification:`*** BTC perpetual contracts often exhibit mean reversion tendencies in their funding rates. Funding rates are designed to bring the contract price in line with the underlying asset's spot price, and deviations from this equilibrium are naturally corrected over time thanks to its core mechanism.

4. ***`Time Homogeneity:`*** The OU process is time-homogeneous, meaning its statistical properties and parameters do not change over time. With a stationary transformed funding rate, the time homogeneity assumption is more likely to hold, further supporting the model's use.

5. ***`Simplicity and Interpretability:`*** The OU process is relatively straightforward to understand and interpret, making it suitable for financial modeling and Monte Carlo simulations.

The solution to the Ornstein-Uhlenbeck SDE is given by:

\begin{align*}
X_t = X_0e^{-\theta t} + \mu(1-e^{-\theta t}) + \sigma \int_0^t e^{-\theta(t-s)} dW_s
\end{align*}

The mean and the variance of this process are given by:

\begin{align*}
E[X_t] = X_0e^{-\theta t} + \mu(1-e^{-\theta t})
\end{align*}

\begin{align*}
V[X_t] = \frac{\sigma^2}{2\theta}(1-e^{-2\theta t})
\end{align*}

One can easily notice that the longterm mean and variance of the OU process are:

\begin{align*}
E[X_\infty] = \underset{t \rightarrow \infty}{lim}~X_0e^{-\theta t} + \mu(1-e^{-\theta t}) = \mu
\end{align*}

\begin{align*}
V[X_\infty] = \underset{t \rightarrow \infty}{lim}~\frac{\sigma^2}{2\theta}(1-e^{-2\theta t}) = \frac{\sigma^2}{2\theta}
\end{align*}

And that is what we will be using for our first approach for parameter estimation.

So this approach relies on the Euler Discretization for the OU SDE since $dX_t \approx X_{t+dt}-X_t$ :

\begin{align*}
X_{t+\Delta t} \approx X_t + \theta(\mu - X_t)\Delta t + \sigma \Delta t Z_t \\
\end{align*}

\begin{equation*}
\textbf{Where } Z_t \sim N(0,1)
\end{equation*}

So by denoting the first difference as $\Delta X_t = X_{t+\Delta t} - X_t$ we get:

\begin{align*}
\Delta X_t \approx \theta(\mu - X_t)\Delta t + \sigma \Delta t Z_t 
\end{align*}

And by rearranging the equation we get the following regression problem:

\begin{align*}
\Delta X_t \approx \theta\mu\Delta t -\theta\Delta t X_t + \sigma \Delta t Z_t = \alpha + \beta X_t + \epsilon_t 
\end{align*}

And by determining the $\beta$ coefficient we get an estimate of $\theta = \frac{-\beta}{\Delta t} $. And for the rest of the parameters $\mu$  and $\sigma$, we know that the transformed funding rate is a stationary standard gaussian. That being said, the longterm mean and variance of the simulated process $X_t$ has to match those statistical properties, hence the equations: 

\begin{align*}   
    \begin{cases}    
        E[X_\infty] = \mu  \\    
        V[X_\infty] = \frac{\sigma^2}{2\theta} \iff \sigma = \sqrt{2\theta .V[X_\infty]}=\sqrt{2\theta}.\sigma_{X_\infty}\\
        \theta = \frac{-\beta}{\Delta t}
    \end{cases}
\end{align*}

In [7]:
def OU_Parameters_Estimation(historical_data, dt=1, display=False):
    
    """
    Estimate the parameters of an Ornstein-Uhlenbeck process using historical data.

    Args:
    - historical_data (pd.Series or np.array): Time series data of the asset prices or returns.
    - dt (float, optional): Time interval between data points. Defaults to 1.
    - display (bool, optional): Whether to display the parameter estimates. Defaults to False.

    Returns:
    - tuple: A tuple containing the estimated parameters of the Ornstein-Uhlenbeck process (mu, sigma, theta).
    """

    dX = historical_data.diff().dropna()

    lagged_X = historical_data.shift().dropna()

    X = np.column_stack((np.ones(len(dX)), lagged_X))
    y = dX.values

    model = sm.OLS(y, X)
    results = model.fit()
    
    theta = -results.params[1]/dt
    mu = historical_data.mean()
    sigma = historical_data.std() * np.sqrt(2*theta)

    if display:
        print(f"*** The Estimates of Ornstein-Uhlenbeck Process Parameters ***\nμ = {mu:.3f}\nσ = {sigma:.3f}\nθ = {theta:.3f}")

    return mu, sigma, theta

To simulate the OU process numerically we used the Euler-Maruyama method and the following discrete-time update rule is applied:

\begin{equation*}
X_{t + \Delta t} = X_t + \theta(\mu - X_t)\Delta t + \sigma \Delta W_t
\end{equation*}

\begin{align*}
& \text{Where:}\\
&X_{t + \Delta t} \text{ is the estimated value of the OU process at the next time step, } t + \Delta t. \\
&X_t \text{ is the current value of the OU process.} \\
&\theta(\mu - X_t)\Delta t \text{ represents the mean-reversion component.} \\
&\sigma \Delta W_t \text{ represents the stochastic component.}
\end{align*}

In [8]:
def simulate_ornstein_uhlenbeck(initial_value, theta, num_steps, num_sim, mu=0, sigma=1,random_shocks=None, dt=1,seed=False):

    """
    Simulate the Ornstein-Uhlenbeck (OU) process using the Euler-Maruyama method.

    Args:
        - initial_value (float): The initial value of the process.
        - theta (float): The mean-reversion parameter of the OU process.
        - num_steps (int): The number of time steps to simulate.
        - num_sim (int): The number of simulations to perform.
        - mu (float, optional): The mean parameter of the OU process. Defaults to 0. 
        - sigma (float, optional): The volatility parameter of the OU process. Defaults to 1.
        - random_shocks (numpy.ndarray, optional): Pre-generated random shocks. Defaults to None.
        - dt (float, optional): The time step between each simulation point. Defaults to 1.
        - seed (int or boolean, optional): Set the seed for reproducibility. Defaults to False.

    Returns:
        - pandas.DataFrame: A DataFrame containing the simulated paths. Each column represents a separate simulation path,
        and the index represents the time steps.
    """

    if seed != False:
        np.random.seed(seed)

    if random_shocks is None:
        random_shocks = np.random.normal(0, np.sqrt(dt), size=(num_steps, num_sim))

    paths = np.zeros((num_steps, num_sim))
    paths[0, :] = initial_value

    for t in range(1, num_steps):
        drift = theta * (mu - paths[t - 1, :]) * dt
        diffusion = sigma * random_shocks[t, :]
        paths[t, :] = paths[t - 1, :] + drift + diffusion

    index = Funding[T0:].index[0:num_steps]    
    paths_df = pd.DataFrame(paths, index=index)

    return paths_df

## 3.2. Simulating the Index Price ($S_t$) Using Merton's Jump Diffusion Process

While examining the index price data, two significant characteristics emerged: continuous diffusion and occasional jumpiness. To effectively represent this intricate behavior, the Merton jump diffusion process was selected for modeling within the Monte Carlo simulation. 

The Merton Jump Diffusion Model describes the dynamics of an asset price, such as BTC in a perpetual futures market, as a combination of continuous diffusion and occasional jumps. The model can be mathematically expressed as follows:

\begin{equation*}
dS_t = \mu S_t dt + \sigma S_t dW_t + dJ_t
\end{equation*}


where:
- $dS_t$ represents the infinitesimal change in the asset price over a small time interval dt.
- $\mu$ is the drift coefficient, representing the expected instantaneous rate of return of the asset.
- $\sigma$ is the volatility coefficient, representing the standard deviation of the asset's returns.
- $dW_t$ is the increment of a Wiener process (Brownian motion) at time t, representing the continuous diffusion component.
- $dJ_t$ is the increment of a jump process at time t, representing the sudden jumps in the asset price.

The jump process $J_t$ is modeled as a compound Poisson process with intensity $\lambda$, where the number of jumps follows a Poisson distribution, and the jump sizes are independently and identically distributed.

The jump sizes can be assumed to follow a log-normal distribution with mean $J_{\text{mean}}$ and standard deviation $J_{\text{std}}$, which accounts for the fact that jumps are often proportional to the asset's current price.

The reasons behind using the `Merton Jump Diffusion Model` for simulating `BTC Perpetual Futures' Index Price` are:

1. ***`Accounting for Jumps:`*** Captures sudden and substantial price movements (jumps) in BTC price, reflecting its high volatility and irregular behavior.

2. ***`Flexibility in Drift and Volatility:`*** Allows separate specification of drift and volatility parameters, enabling adaptation to different market conditions and historical characteristics of BTC.

3. ***`Incorporating Market Sentiment and News Impact:`*** Includes exogenous events like market sentiment, regulatory news, and macroeconomic factors, which can trigger significant jumps in BTC price.

By incorporating both the continuous diffusion and jump components, the Merton Jump Diffusion Model provides a comprehensive representation of the asset price dynamics, capturing the sudden jumps and heavy-tailed distributions observed in cryptocurrency markets like BTC perpetual futures.

The parameter estimation method for Merton's jump diffusion model is as follows. We start by computing logarithmic returns within index price sequences $S_t$:

\begin{equation*}
\text{Logarithmic Return: } R_t = \log\left(\frac{S_t}{S_{t-1}}\right) \text{for } t \in [1,n]
\end{equation*}

And then we determine the continuous component's parameters which is governed by a Geometric Brownian Motion with a drift parameter $\mu_{\text{gbm}}$ and a volatility parameter $\sigma_{\text{gbm}}$, using $R_t$:

\begin{equation*}
\mu_{\text{gbm}} = \frac{1}{n} \sum_{t=1}^{n} R_t, \quad \sigma_{\text{gbm}} =  \sqrt{\frac{1}{n}\sum_{t=1}^{n} (R_t - \mu_{\text{gbm}})^2}
\end{equation*}

By subtracting the continuous drift component from the logarithmic returns, we isolate the effects of jump events and we denote it as $C_t$:

\begin{equation*}
\text{Jump Size: } C_t = R_t - \mu_{\text{gbm}}
\end{equation*}

And then using the jumps, we estimate the mean $\mu_{\text{jump}}$ and volatility $\sigma_{\text{jump}}$ that intricately govern these ascending perturbations:

\begin{equation*}
\mu_{\text{jump}} = \frac{1}{n} \sum_{t=1}^n C_t, \quad \sigma_{\text{jump}} = \sqrt{\frac{1}{n}\sum_{t=1}^n (C_t - \mu_{\text{jump}})^2}
\end{equation*}

And finally we provide an estimate of the jump intensity $\lambda$ using the method of moments. We employ enumeration of jumps exceeding a specified threshold (determined via visual inspection of the training set jumps), rendering the jump intensity as the ratio of such jumps to the entirety of the observation time:

\begin{equation*}
\lambda =\frac{\text{Number of Jumps}}{\text{Observation Time}} =\frac{N_\text{Jumps}}{n \cdot \Delta t}
\end{equation*}

In [9]:
def Merton_Parameters_Estimation(X, jump_threshold, dt=1, display=False): 

    """
    Estimate Merton model parameters.
    
    Args:
        - X (pandas.Series): Input data.
        - jump_threshold (float): Threshold for identifying jumps.
        - dt (float, optional): Time step. Defaults to 1.
        - display (bool, optional): Whether to display parameter estimates. Defaults to False.
    
    Returns:
        - tuple: Tuple containing Merton model parameters (mu_gbm, sigma_gbm, jump_intensity, jump_mean, jump_std).
    """

    warnings.simplefilter("ignore")

    log_returns = np.log(X / X.shift(1)).dropna()

    mu_gbm = log_returns.mean() / dt
    sigma_gbm = log_returns.std() / np.sqrt(dt)

    log_jump_sizes = log_returns - mu_gbm * dt

    jump_mean = log_jump_sizes.mean() / dt
    jump_std = log_jump_sizes.std() / np.sqrt(dt)

    jumps = X.diff() > jump_threshold
    num_jumps = np.sum(jumps)
    jump_intensity = num_jumps/(len(X)*dt)

    if display:
        print("*** The Estimates of Merton Model Parameters ***")
        print(f"GBM Parameters:")
        print(f"μ (Drift) = {mu_gbm:.3f}")
        print(f"σ (Volatility) = {sigma_gbm:.3f}")
        print(f"\nJump Parameters:")
        print(f"Jump Intensity (λ) = {jump_intensity:.3f}")
        print(f"μ_jump (Jump Mean) = {jump_mean:.3f}")
        print(f"σ_jump (Jump Volatility) = {jump_std:.3f}")

    return mu_gbm, sigma_gbm, jump_intensity, jump_mean, jump_std

To simulate Merton's Jump Diffusion Process numerically using the Euler-Maruyama method, the following discrete-time update rule is applied:

\begin{equation}
S_{t + \Delta t} = S_t+ \mu S_t \Delta t + \sigma S_t \Delta W_t + \Delta J_t
\end{equation}

Here's how this relates to the simulation code:

\begin{align*}
&S_{t + \Delta t} \text{ is the estimated value of the process at the next time step, } t + \Delta t. \\
&S_t \text{ is the current value of the process.} \\
&\mu S_t \Delta t \text{ represents the deterministic drift component.} \\
&\sigma S_t \Delta W_t \text{ represents the stochastic component due to Brownian motion.} \\
&\Delta J_t \text{ represents the impact of jumps on the process.}
\end{align*}

In [10]:
def simulate_merton(initial_value, r, sigma, lam, mu_jump, sigma_jump, num_steps=None, num_sim=None, random_shocks = None, dt=1, seed=False):
    
    """
    Simulate the Merton Model (Geometric Brownian Motion + Jump Diffusion) using the Euler-Maruyama method.

    Args:
        - initial_value (float): The initial value of the process.
        - r (float): The drift (mean) parameter of the GBM process.
        - sigma (float): The volatility (standard deviation) parameter of the GBM process.
        - lam (float): The jump intensity parameter, representing the average number of jumps per unit time.
        - mu_jump (float): The jump mean parameter, representing the average size of the jumps.
        - sigma_jump (float): The jump volatility parameter, representing the standard deviation of jump sizes.
        - num_steps (int, optional): The number of time steps to simulate. Defaults to None.
        - num_sim (int, optional): The number of simulations to perform. Defaults to None.
        - random_shocks (numpy.ndarray, optional): Pre-generated random shocks. Defaults to None.
        - dt (float, optional): The time step between each simulation point. Defaults to 1.
        - seed (int or bool, optional): Set the seed for reproducibility. Defaults to False.

    Returns:
        - pandas.DataFrame: A DataFrame containing the simulated paths. Each column represents a separate simulation path,
        and the index represents the time steps.
    """

    if seed != False:
        np.random.seed(seed)

    if random_shocks is None:
        random_shocks_gbm = np.random.normal(0, 1, size=(num_steps, num_sim))
    else:
        random_shocks_gbm = random_shocks
        
    random_shocks_jump = np.random.poisson(lam * dt, size=(num_steps, num_sim))

    paths = np.zeros((num_steps, num_sim))
    paths[0, :] = initial_value

    for t in range(1, num_steps):
        drift_gbm = (r - 0.5 * sigma ** 2) * dt
        diffusion_gbm = sigma * np.sqrt(dt) * random_shocks_gbm[t, :]
        paths[t, :] = paths[t - 1, :] * np.exp(drift_gbm + diffusion_gbm)

        jumps = mu_jump * random_shocks_jump[t, :] + sigma_jump * np.sqrt(random_shocks_jump[t, :])
        paths[t, :] *= np.exp(jumps)

    index = [(datetime.strptime(T0,'%Y-%m-%d') + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(num_steps)]
    paths_df = pd.DataFrame(paths, index=index)

    return paths_df

## 3.3. Deriving the Liquidation Price ($S_t^L$), the Expected Liquidation Time ($E(\tau)$) and the Probability of Liquidation Before T ($P(\tau \le T)$)

Let $\tau$ be the liquidation time. To keep the we mentioned earlier position open, the following inequality must be true:

\begin{equation*}
PnL_t + M_t \ge U_t
\end{equation*}

Where:

\begin{align*} 
    PnL_t = 
    \begin{cases}
        NL(\frac{1}{S_0}-\frac{1}{S_t}) \text{ for Long Position Holders}\\
        NL(\frac{1}{S_t}-\frac{1}{S_0}) \text{ for Short Position Holders}
    \end{cases} 
    \text{ is the Profit \& Loss at time t.}
\end{align*}

\begin{equation*}
    M_t = 
    \begin{cases}
        \frac{N}{S_0}-3\sum_{i=1}^t\frac{NL}{S_i}r_i \text{ for Long Position Holders}\\
        \frac{N}{S_0}+3\sum_{i=1}^t\frac{NL}{S_i}r_i \text{ for Short Position Holders}
    \end{cases}
    \text{ is the Wallet Balance at time t.}
\end{equation*}

\begin{equation*}
    U_t \text{ is the Maintenance Margin at time t and it is unique to each exchange.}
\end{equation*}

So we can easily notice that:

\begin{equation*}
\tau = inf\{t\ge0|PnL_t + M_t < U_t\}
\end{equation*}

And that the liquidation threshold is given by:

\begin{equation*}
PnL_t + M_t = U_t
\end{equation*}

This equation allows us to derive the liquidation price for both positions:

\begin{equation*}
    S_t^L = 
    \begin{cases}
        \frac{L^2S_0}{L^2(1-3\sum_{i=1}^t\frac{S_0}{S_i}r_i)+L(1-\frac{S_0}{N}U_t)}\text{ for Long Position Holders}\\
        \frac{L^2S_0}{L^2(1-3\sum_{i=1}^t\frac{S_0}{S_i}r_i)-L(1-\frac{S_0}{N}U_t)}\text{ for Short Position Holders}
    \end{cases}
\end{equation*}

And we can rewrite $\tau$ as: 

\begin{equation*}
    \tau=
    \begin{cases}
        inf\{t\ge0|S_t<S_t^{L,~long}~\}\text{ for Long Position Holders} \\
        inf\{t\ge0|S_t>S_t^{L,~short}\}\text{ for Short Position Holders}
    \end{cases}
\end{equation*}

Following the simulation of the funding rate and index price, the focus shifts to calculating the time series of liquidation prices for each scenario. By monitoring the index's behavior relative to these liquidation prices, liquidation times are determined when the index breaches the specific threshold for that specific position. The aggregation of these times, coupled with averaging across each path (which leverages the `Central Limit Theorem`), allows us to derive the `expected liquidation time` $E(\tau)$ for both long and short positions separately. 

The computation of the `probability of liquidation before a specific time T` $P(\tau \le T)$ follows a straightforward process. It involves dividing the number of liquidated paths before T by the total number of simulations performed. This ratio effectively quantifies the likelihood of encountering a liquidation event within the designated timeframe.

In [11]:
def LiquidationPrice(allS, allR, L, position, exchange, crrntIndex=num_steps):
    
    """
    Calculate the liquidation price for given position and leverage in a specific exchange.

    Args:
        - allS (array-like): Array of index prices.
        - allR (array-like): Array of funding rates.
        - L (float): Leverage.
        - position (str): Position type ('long' or 'short').
        - exchange (str): Exchange name ('BINANCE', 'DERIBIT', 'BYBIT', or 'OKEX').
        - crrntIndex (int, optional): Current index. Defaults to num_steps.

    Returns:
        - float: Liquidation price.

    Raises:
        - ValueError: If the `position` is neither 'long' nor 'short'.
        - ValueError: If the `exchange` is not one of 'BINANCE', 'DERIBIT', 'BYBIT', or 'OKEX'.
    """
    
    k = 3 * np.sum(allR[1:crrntIndex] * S0 / allS[1:crrntIndex])

    if exchange.upper() == 'BINANCE': 
        if position.upper() == 'LONG':  
            return max(((L ** 2) * S0) / ((L ** 2) * (1 - k) + L * (0.9955) - 0.0005), 0)
        elif position.upper() == 'SHORT':
            return max(((L ** 2) * S0) / ((L ** 2) * (1 - k) - L * (0.9955) + 0.0005), 0)
        else:
            raise ValueError("Position is either 'long' or 'short'")

    elif exchange.upper() == 'DERIBIT':
        if position.upper() == 'LONG':  
            return max(((L ** 2) * S0) / ((L ** 2) * (1 - k) + L * (0.9895 - 0.005 * N / (100 * S0)) - 0.0005), 0)
        elif position.upper() == 'SHORT':
            return max(((L ** 2) * S0) / ((L ** 2) * (1 - k) - L * (0.9895 - 0.005 * N / (100 * S0)) + 0.0005), 0)
        else:
            raise ValueError("Position is either 'long' or 'short'")

    elif exchange.upper() == 'BYBIT':
        if position.upper() == 'LONG':  
            return max(((L ** 2) * S0) / ((L ** 2) * (1 - k) + L * (0.99445) - 0.00055), 0)
        elif position.upper() == 'SHORT':
            return max(((L ** 2) * S0) / ((L ** 2) * (1 - k) - L * (0.99445) + 0.00055), 0)
        else:
            raise ValueError("Position is either 'long' or 'short'")

    elif exchange.upper() == 'OKEX':
        if position.upper() == 'LONG':  
            return max(((L ** 2) * S0) / ((L ** 2) * (1 - k) + L * (0.995) - 0.001), 0)
        elif position.upper() == 'SHORT':
            return max(((L ** 2) * S0) / ((L ** 2) * (1 - k) - L * (0.995) + 0.001), 0)
        else:
            raise ValueError("Position is either 'long' or 'short'")
    else:
        raise ValueError("Exchange should be one of 'BINANCE', 'DERIBIT', 'BYBIT', or 'OKEX'")

In [12]:
def Liquidation(funding_rate, L, position, exchange, display=True, dt=1):

    """
    Calculate the expected liquidation time and probability of liquidation for given position and leverage in a specific exchange.

    Args:
        - funding_rate (pandas.DataFrame): DataFrame containing simulated funding rates for different paths.
        - L (float): Leverage.
        - position (str): Position type ('long' or 'short').
        - exchange (str): Exchange name.
        - display (bool, optional): Whether to display the results. Defaults to True.
        - dt (float, optional): Time step between each simulation point. Defaults to 1.

    Returns:
        - tuple: A tuple containing expected liquidation time, probability of liquidation, number of liquidated paths,
        and list of liquidation times for each path.

    Raises:
        - ValueError: If the `position` is neither 'long' nor 'short'.
        - ValueError: If the `exchange` is not one of 'BINANCE', 'DERIBIT', 'BYBIT', or 'OKEX'.
    """

    lqPrice = 0
    lqTime = []
    lq = 0 # Number of path that are being liquidated before T
    
    for path in simulated_index.columns:        
        for i in range (1, len(simulated_index[path])):

            lqPrice = LiquidationPrice(simulated_index[path].to_numpy(), funding_rate[path].to_numpy()/100, L, position, exchange, i)
            
            if (position.upper() == 'LONG') and (simulated_index[path].iloc[i] < lqPrice):
                lqTime.append(i*dt)
                lq += 1
                break

            if (position.upper() == 'SHORT') and (simulated_index[path].iloc[i] > lqPrice):
                lqTime.append(i*dt)
                lq += 1
                break

    ExpLqTime = np.average(lqTime) # if τ <= T, calculate E(τ)

    # Probability of getting liquidated (at least once) in T time periods
    LqProb = lq / num_sim
    
    if display:
        print(f"*** SIMULATED PERPETUALS CONTRACT REPORT ***\nPosition Entry Date: {T0}\nArtificial Maturity (Trader's Exit): {T}\nPosition Specification: {position}\nExchange Platform: {exchange}\nNumber of Purchased Contracts: {N}\nLeverage: {L}x\nInitial Position Value: Ƀ {N*L/S0:.5f}\nExpected Liquidation Time: {ExpLqTime:.1f} Days\nProbability of Getting Liquidated Before Artificial Maturity: {LqProb:.3f}")
    
    return ExpLqTime, LqProb, lq, lqTime

# 4. The Pipeline & Results Visualization

## 4.1. Cleaning & Simulating the Funding Rates ($r_t$)

In [13]:
FundingDict = dict()

FundingCopy = Funding.copy()

T0_obj = datetime.strptime(T0, '%Y-%m-%d')
prior_date_obj = T0_obj - timedelta(days = 7 * 30)
prior_date = prior_date_obj.strftime('%Y-%m-%d')

for col in Funding.columns:

    
    FundingCol = IQR_Cleaning(FundingCopy[col],2,2)[prior_date:]
    mu, sigma, theta = OU_Parameters_Estimation(FundingCol[:T0], dt=1, display=Display)

    r0 = FundingCol.loc[T0]
    time_steps = FundingCol[T0:T].shape[0] 
    
    FundingDict[col] = simulate_ornstein_uhlenbeck(r0, theta, num_steps, num_sim,mu=mu, sigma=sigma,random_shocks=random_shocks)

    if Display:
        print(f'*** {col} ***\n')
        print(f"\nThe average standard deviation of simulated paths: {FundingDict[col].std().mean():.3f}")
        print(f"The average mean of simulated paths: {FundingDict[col].mean().mean():.3f}")

    if Visualize:
        fig = go.Figure()

        path_colors = ['lightseagreen', 'red', 'orange', 'purple', 'tan', 'magenta', 'cyan', 'lightgreen', 'pink', 'lightsalmon']
        
        fig.add_trace(go.Scatter(x=FundingCol[:T0].index, y=FundingCol[:T0], mode='lines', name='Funding Training Set'))
        fig.add_trace(go.Scatter(x=FundingCol[T0:T].index, y=FundingCol[T0:T], mode='lines', name='Funding Test Set',line=dict(color='navy')))
        for i, column in enumerate(FundingDict[col].iloc[:, :10].columns):
            fig.add_trace(go.Scatter(x=FundingCol[T0:T].index, y=FundingDict[col][column], mode='lines', name=f'Simulated Path {column+1}',line=dict(color=path_colors[i])))

        fig.update_layout(
            title=rf"Monte Carlo Simulation of Rate Using Ornstein-Uhlenbeck on Test Set | Market: {col}",
            xaxis_title="Time Steps",
            yaxis_title="Funding Rate",
            height=600,
            title_x=0.5
        )

        fig.show()

# 4.2. Simulating the Index Price ($S_t$)

In [14]:
x = Merton_Parameters_Estimation(IndexPrice.loc[IndexPrice.index <= T0],2000, dt=1, display=Display)
simulated_index = simulate_merton(initial_value=S0, r=x[0], sigma=x[1],  lam=x[2], mu_jump=x[3], sigma_jump=x[4], num_steps=num_steps, num_sim=num_sim, dt=1,random_shocks=random_shocks)

if Visualize:
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=IndexPrice.loc[IndexPrice.index <= T0].index, y=IndexPrice.loc[IndexPrice.index <= T0], mode='lines', name='Index Price Training Set'))
    fig.add_trace(go.Scatter(x=IndexPrice[T0:T].index, y=IndexPrice[T0:T], mode='lines', name='Index Price Test Set', line=dict(color='navy')))

    for i, column in enumerate(simulated_index.iloc[:,:10].columns):
        fig.add_trace(go.Scatter(x=IndexPrice[T0:T].index, y=simulated_index[column], mode='lines', name=f'Simulated Path {column + 1}',line=dict(color=path_colors[i])))
            
    fig.update_layout(
            title=rf"Monte Carlo Simulation of Index Price Using Merton's Jump Diffusion Model",
            xaxis_title="Time Steps",
            yaxis_title="Index Price ($)",
            height=600,
            title_x=0.5
    )

    fig.show()

## 4.3. Simulating the Liquidation Process and Deriving the Liquidation Price ($S_t^L$), the Expected Liquidation Time ($E(\tau)$) and the Probability of Liquidation Before T ($P(\tau \le T)$)

In [15]:
liq = Liquidation(FundingDict[Exchange], L, Position, Exchange)

*** SIMULATED PERPETUALS CONTRACT REPORT ***
Position Entry Date: 2023-01-21
Artificial Maturity (Trader's Exit): 2023-04-01
Position Specification: Short
Exchange Platform: OKEX
Number of Purchased Contracts: 1
Leverage: 50x
Initial Position Value: Ƀ 0.00221
Expected Liquidation Time: 7.5 Days
Probability of Getting Liquidated Before Artificial Maturity: 0.896


In [16]:
end1 = time.time()
elapsed_time1 = end1 - start
print(f"Elapsed Time: {int(elapsed_time1 // 3600):0>2}:{int((elapsed_time1 % 3600) // 60):0>2}:{elapsed_time1 % 60:05.2f}")

Elapsed Time: 00:00:07.32


In [17]:
ExpLDict = dict()
ExpSDict = dict()

for Ex, maxL in [('BINANCE',125), ('BYBIT',100), ('DERIBIT',50), ('OKEX',125)]:

    ExpL = []
    ProbL = []
    ExpS = []
    ProbS = []
    
    for l in range(maxL):
        liqL = Liquidation(FundingDict[Ex],l+1,'long',Ex,display=False)
        ExpL.append(liqL[0])
        ProbL.append(liqL[1])
        liqS = Liquidation(FundingDict[Ex],l+1,'short',Ex,display=False)
        ExpS.append(liqS[0])
        ProbS.append(liqS[1])

    ExpLDict[Ex] = ExpL
    ExpSDict[Ex] = ExpS

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=list(range(1, maxL+1)), y=ExpL, mode='lines', name='E(' + '\u03C4' + ')',showlegend=True,yaxis='y'))
    fig.add_trace(go.Scatter(x=list(range(1, maxL+1)), y=ProbL, mode='lines', name='P('+'\u03C4\u2264'+'T)', yaxis='y2'))

    fig.update_layout(
        title=f'Expected Liquidation Time & Probability of Liquidation Before T = {T} for Long Position | Number of Simulations : {num_sim} | Market : {Ex}',
        xaxis_title='Leverage',
        yaxis_title='Expected Liquidation Time (Days)',
        yaxis2=dict(title='Probability of Liquidation Before T', overlaying='y', side='right'),
        height=550,
        title_x=0.5
    )

    fig.show()
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=list(range(1, maxL+1)), y=ExpS, mode='lines', name='E(' + '\u03C4' + ')',showlegend=True,yaxis='y'))
    fig.add_trace(go.Scatter(x=list(range(1, maxL+1)), y=ProbS, mode='lines', name='P('+'\u03C4\u2264'+'T)', yaxis='y2'))

    fig.update_layout(
        title=f'Expected Liquidation Time & Probability of Liquidation Before T = {T} for Short Position | Number of Simulations : {num_sim} | Market : {Ex}',
        xaxis_title='Leverage',
        yaxis_title='Expected Liquidation Time (Days)',
        yaxis2=dict(title='Probability of Liquidation Before T', overlaying='y', side='right'),
        height=550,
        title_x=0.5
    )

    fig.show()

In [18]:
end2 = time.time()
elapsed_time2 = end2 - start
print(f"Elapsed Time: {int(elapsed_time2 // 3600):0>2}:{int((elapsed_time2 % 3600) // 60):0>2}:{elapsed_time2 % 60:05.2f}")

Elapsed Time: 00:12:54.67


The analysis of the liquidation pipeline’s outcomes reveals significant in sights into the impact of leverage on risk and liquidation dynamics. As leverage increases, the inherent risk amplifies for both long and short positions. 

This heightened risk is manifested by a decrease in the expected liquidation time as leverage grows. Moreover, the probability of liquidation before the predefined maturity time exhibits an upward trend with escalating leverage.

Notably, the relationship between leverage and expected liquidation time is characterized by a convex pattern, where the rate of decrease in expected liquidation time accelerates with higher leverage. Conversely, the probability of liquidation before maturity displays a concave behavior, indicating that the incremental increase in leverage leads to a diminishing rise in the likelihood of early liquidation.
 
These findings underscore the intricate interplay between leverage, risk, and liquidation metrics, providing traders with valuable insights for optimizing their positions and strategies in the perpetual contracts market.

# 5. Backtesting

In [19]:
Index_test = IndexPrice.loc[IndexPrice.index >= T0]
Funding_test = Funding[Exchange].loc[Funding.index >= T0]

fig = go.Figure()

short_liq_time = dict()
long_liq_time = dict()

for j, Lev in enumerate(sorted(list(set([5, 10, 20, L, MaxL[Exchange]])))):
    LongBoundary = []

    for i in range(num_steps):
        S1 = LiquidationPrice(Index_test.to_numpy(), Funding_test.to_numpy()/100, Lev, 'long',Exchange, i)
        LongBoundary.append(S1)

        if Index_test.iloc[i] < S1:
            long_liq_time[Lev] = i+1
            break

    fig.add_trace(go.Scatter(x=Index_test.index[0:num_steps], y=LongBoundary, mode='lines', name=f"{Lev}x Leverage (Long)", line=dict(color=path_colors[j]), legendgroup=f"{Lev}x Leverage"))

for j, Lev in enumerate(sorted(list(set([5, 10, 20, L, MaxL[Exchange]])))):
    ShortBoundary = []

    for i in range(num_steps):
        S2 = LiquidationPrice(Index_test.to_numpy(), Funding_test.to_numpy()/100, Lev, 'short',Exchange, i)
        ShortBoundary.append(S2)

        if Index_test.iloc[i] > S2:
            short_liq_time[Lev] = i+1
            break
    
    fig.add_trace(go.Scatter(x=Index_test.index[0:num_steps], y=ShortBoundary, mode='lines', name=f"{Lev}x Leverage (Short)", line=dict(color=path_colors[j], dash='dot'), legendgroup=f"{Lev}x Leverage"))

fig.add_trace(go.Scatter(x=Index_test.index[0:num_steps], y=Index_test, mode='lines', name="Index Price"))

fig.update_layout(
    title=f'Position Boundaries on Test Set | {Exchange}',
    xaxis_title='Time',
    yaxis_title='Index Price ($)',
    height=550,
    title_x=0.5
)
liq_time = {'SHORT':short_liq_time,'LONG':long_liq_time}

fig.show()
      
print(f"*** REAL PERPETUALS CONTRACT REPORT ***\nPosition Entry Date: {T0}\nArtificial Maturity (Trader's Exit): {T}\nPosition Specification: {Position}\nExchange Platform: {Exchange}\nNumber of Purchased Contracts: {N}\nLeverage: {L}x\nInitial Position Value: Ƀ {N*L/S0:.5f}\nActual Liquidation Time: {liq_time[Position.upper()][L]} Days")



*** REAL PERPETUALS CONTRACT REPORT ***
Position Entry Date: 2023-01-21
Artificial Maturity (Trader's Exit): 2023-04-01
Position Specification: Short
Exchange Platform: OKEX
Number of Purchased Contracts: 1
Leverage: 50x
Initial Position Value: Ƀ 0.00221
Actual Liquidation Time: 10 Days


In [20]:
for Ex, maxL in [('BINANCE', 125), ('BYBIT', 100), ('DERIBIT', 50), ('OKEX', 125)]:

    Index_test = IndexPrice.loc[IndexPrice.index >= T0]
    Funding_test = Funding[Ex].loc[Funding.index >= T0]

    short_liq_time = []
    long_liq_time = []

    for Lev in range(1,maxL+1):
        
        for i in range(num_steps):
            S1 = LiquidationPrice(Index_test.to_numpy(), Funding_test.to_numpy()/100, Lev, 'long',Ex, i)
            
            if Index_test.iloc[i] < S1:
                long_liq_time.append(i+1)
                break

        for i in range(num_steps):
            S2 = LiquidationPrice(Index_test.to_numpy(), Funding_test.to_numpy()/100, Lev, 'short',Ex, i)
            
            if Index_test.iloc[i] > S2:
                short_liq_time.append(i+1)
                break
    
    if len(long_liq_time)==0:
        long_liq_time = [num_steps]*maxL

    if len(short_liq_time)==0:
        short_liq_time = [num_steps]*maxL

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=list(range(1,maxL+1)), y=long_liq_time, mode='lines', name='\u03C4',showlegend=True))
    fig.add_trace(go.Scatter(x=list(range(1,maxL+1)), y=ExpLDict[Ex], mode='lines', name='E(\u03C4)',showlegend=True))

    fig.update_layout(
        title=f'Actual Liquidation Time \u03C4 VS Expected Liquidation Time E(\u03C4) for Long Position | {Ex}',
        xaxis_title='Leverage',
        yaxis_title='Liquidation Time (Days)',
        height=550,
        title_x=0.5
    )
    
    fig.show()

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=list(range(1,maxL+1)), y=short_liq_time, mode='lines', name='\u03C4',showlegend=True))
    fig.add_trace(go.Scatter(x=list(range(1,maxL+1)), y=ExpSDict[Ex], mode='lines', name='E(\u03C4)',showlegend=True))

    fig.update_layout(
        title=f'Actual Liquidation Time \u03C4 VS Expected Liquidation Time E(\u03C4) for Long Position | {Ex}',
        xaxis_title='Leverage',
        yaxis_title='Liquidation Time (Days)',
        height=550,
        title_x=0.5
    )
    
    fig.show()

The backtesting phase of the liquidation pipeline validates its effectiveness and sheds light on its key strengths and potential improvements. Notably, the comparison between the plots of expected and actual liquidation times reaffirms the accuracy of our methodology. Both plots exhibit similar trends and maintain the same scale, pointing out good insights in this approach. 

The backtesting results also underscore the relative performance of the long and short positions. While the short side demonstrates a notably successful outcome, the long side exhibits decent but not so accurate performance. Recognizing this discrepancy, the primary avenue for future enhancement lies in refining the plots related to the long position.

# 6. Conclusion


This work provides a robust tool for risk assessment, strategy evaluation, and decision-making in perpetual contracts trading. By estimating expected liquidation time and probability of liquidation before maturity, it helps traders navigate cryptocurrency markets effectively. Future enhancements include refining long position simulations, customizing funding rates' models for different exchanges, and automating data acquisition for real-time market insights. These improvements will make the pipeline indispensable for traders in perpetual contracts trading.